## Set-up

In [27]:
import torch
import numpy as np
import cv2
import matplotlib.pyplot as plt
import sys
sys.path.append("..")
from segment_anything import sam_model_registry, SamPredictor
from segment_anything.utils.onnx import SamOnnxModel

import onnxruntime
from onnxruntime.quantization import QuantType
from onnxruntime.quantization.quantize import quantize_dynamic

## Export an ONNX model

In [3]:
import torch
import sys
sys.path.append("..")
from segment_anything.modeling.image_encoder import ImageEncoderViT

checkpoint = "../savedmodel/sam_vit_h_4b8939.pth"
model_type = "vit_h"
sam = sam_model_registry[model_type](checkpoint=checkpoint)
# print(sam.image_encoder)

from functools import partial

image_encoder=ImageEncoderViT(
    depth=32,
    embed_dim=1280,
    img_size=1024,
    mlp_ratio=4,
    norm_layer=partial(torch.nn.LayerNorm, eps=1e-6),
    num_heads=16,
    patch_size=16,
    qkv_bias=True,
    use_rel_pos=True,
    global_attn_indexes=[7, 15, 23, 31],
    window_size=14,
    out_chans=256,
)

####################################################################

# Load the weights from sam's image_encoder
image_encoder.load_state_dict(sam.image_encoder.state_dict())
image_encoder.eval()

# save image_encoder model
torch.save(image_encoder.state_dict(), "../savedmodel/ImageEncoderViT.pth")


In [5]:
# load image_encoder model
image_encoder=ImageEncoderViT(
    depth=32,
    embed_dim=1280,
    img_size=1024,
    mlp_ratio=4,
    norm_layer=partial(torch.nn.LayerNorm, eps=1e-6),
    num_heads=16,
    patch_size=16,
    qkv_bias=True,
    use_rel_pos=True,
    global_attn_indexes=[7, 15, 23, 31],
    window_size=14,
    out_chans=256,
)
image_encoder.load_state_dict(torch.load("../savedmodel/ImageEncoderViT.pth"))
image_encoder.eval()

# An example input you would normally provide to your model's forward() method.
example_input = {"input": torch.randn(1, 3, 1024, 1024)}  # adjust as necessary

dynamic_axes = {
    "point_coords": {1: "num_points"},
    "point_labels": {1: "num_points"},
}

import warnings

with warnings.catch_warnings():
    warnings.filterwarnings("ignore", category=torch.jit.TracerWarning)
    warnings.filterwarnings("ignore", category=UserWarning)
    with open("../savedmodel/ImageEncoderViT.onnx", "wb") as f:
        torch.onnx.export(
            image_encoder,
            example_input.values(),
            "../savedmodel/ImageEncoderViT.onnx",
            export_params=True,
            verbose=False,
            opset_version=14,
            do_constant_folding=True,
            input_names=example_input.keys(),
            output_names=["output"],
            dynamic_axes=dynamic_axes,
        )

window_size 14
window_size 14
window_size 14
window_size 14
window_size 14
window_size 14
window_size 14
window_size 14
window_size 14
window_size 14
window_size 14
window_size 14
window_size 14
window_size 14
window_size 14
window_size 14
window_size 14
window_size 14
window_size 14
window_size 14
window_size 14
window_size 14
window_size 14
window_size 14
window_size 14
window_size 14
window_size 14
window_size 14
================ Diagnostic Run torch.onnx.export version 2.0.1 ================
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



In [28]:
# quantize
encode_qt_path = "../savedmodel/ImageEncoderViT_qt.onnx"
quantize_dynamic("../savedmodel/ImageEncoderViT.onnx", encode_qt_path, weight_type=QuantType.QUInt8)

Ignore MatMul due to non constant B: /[/blocks.0/attn/MatMul]
Ignore MatMul due to non constant B: /[/blocks.0/attn/MatMul_1]
Ignore MatMul due to non constant B: /[/blocks.1/attn/MatMul]
Ignore MatMul due to non constant B: /[/blocks.1/attn/MatMul_1]
Ignore MatMul due to non constant B: /[/blocks.2/attn/MatMul]
Ignore MatMul due to non constant B: /[/blocks.2/attn/MatMul_1]
Ignore MatMul due to non constant B: /[/blocks.3/attn/MatMul]
Ignore MatMul due to non constant B: /[/blocks.3/attn/MatMul_1]
Ignore MatMul due to non constant B: /[/blocks.4/attn/MatMul]
Ignore MatMul due to non constant B: /[/blocks.4/attn/MatMul_1]
Ignore MatMul due to non constant B: /[/blocks.5/attn/MatMul]
Ignore MatMul due to non constant B: /[/blocks.5/attn/MatMul_1]
Ignore MatMul due to non constant B: /[/blocks.6/attn/MatMul]
Ignore MatMul due to non constant B: /[/blocks.6/attn/MatMul_1]
Ignore MatMul due to non constant B: /[/blocks.7/attn/MatMul]
Ignore MatMul due to non constant B: /[/blocks.7/attn/Ma